# Allometric equation uncertainty

In [ ]:
import allometry
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

In [ ]:
# Load harvest data from Chave et al 2014
har = pd.read_csv("../Chave_harvest_db/Chave_GCB_Direct_Harvest_Data.csv")
loc = pd.read_csv("../Chave_harvest_db/Localities.csv")
loc['Forest_type'] = loc.Forest_type.str.replace(' forest','')
print har.columns
print loc.columns

In [ ]:
# Select harvest data from South America
sa = loc[loc.Locality.str.contains('Brazil') | loc.Locality.str.contains('Colombia') |
         loc.Locality.str.contains('Venezuela') | loc.Locality.str.contains('Peru') |
         loc.Locality.str.contains('French Guiana') | loc.Locality.str.contains('Costa Rica')
        ]['Abbreviation'].tolist()

sahar = har[har.Site.isin(sa)]
sahar.Site.unique()

In [ ]:
# allometric functions
def est_alvarez(row):
    return allometry.alvarez(row.DBH_cm, row.Gravity, row.Holdridge)

def est_chaveI(row):
    fo = loc[loc.Abbreviation == row.Site]['Forest_type'].item()
    return allometry.chaveI(row.DBH_cm, row.Gravity, fo)

def est_chaveI_or(row):
    fo = loc[loc.Abbreviation == row.Site]['Forest_type'].item()
    return allometry.chaveI_original(row.DBH_cm, row.Gravity, fo)

def est_E(row):
    #E = ( 0.178 × TS-0.938 × CWD-6.61× PS ) ×10 −3
    ts = loc[loc.Abbreviation == row.Site]['Temp_Seasonality'].item()
    cwd = loc[loc.Abbreviation == row.Site]['CWD_mm_yr'].item()
    ps = loc[loc.Abbreviation == row.Site]['Precip_Seasonality_perc'].item()
    E = (0.178 * ts - 0.938 * cwd - 6.61 * ps) * 1e-3
    return E

def est_holdr(row):
    alt = loc[loc.Abbreviation == row.Site]['Altitude'].item()
    prep = loc[loc.Abbreviation == row.Site]['Mean_Annual_Precip'].item()
    return allometry.holdridge_col(alt, prep)

def est_chaveII(row):
    return allometry.chaveII(row.DBH_cm, row.Gravity, e_value = row.E)

def est_chaveII_dh(row):
    return allometry.chaveII_dh(row.DBH_cm, row.Total_height_m, row.Gravity)

In [ ]:
def logdh(diam, height, density):
    return np.log(0.0673) + 0.976 * (np.log(density) + np.log(height) + 2 * np.log(diam))

In [ ]:
har['E'] = har.apply(est_E, axis=1)
sahar.loc[:,'Holdridge'] = sahar.apply(est_holdr, axis=1)

har['ChaveI'] = har.apply(est_chaveI, axis=1)
har['ChaveI_or'] = har.apply(est_chaveI_or, axis=1)
har['ChaveII'] = har.apply(est_chaveII, axis=1)
har['ChaveII_dh'] = har.apply(est_chaveII_dh, axis=1)
sahar.loc[:,'Alvarez'] = sahar.apply(est_alvarez, axis=1)

In [ ]:
# Estimate errors
har['ChaveI_error'] = (np.log(har.ChaveI) - np.log(har.AGB_kg))**2
har['ChaveI_or_error'] = (np.log(har.ChaveI_or) - np.log(har.AGB_kg))**2
har['ChaveII_error'] = (np.log(har.ChaveII) - np.log(har.AGB_kg))**2
har['ChaveII_dh_error'] = (np.log(har.ChaveII_dh) - np.log(har.AGB_kg))**2
sahar.loc[:,'Alvarez_error'] = (np.log(sahar.Alvarez) - np.log(sahar.AGB_kg))**2

In [ ]:
# SEEs
print (har.ChaveI_error.sum() / (har.shape[0]-4))**0.5
print (har.ChaveI_or_error.sum() / (har.shape[0]-4))**0.5
print (har.ChaveII_error.sum() / (har.shape[0]-5))**0.5
print (har.ChaveII_dh_error.sum() / (har.shape[0]-2))**0.5
print (sahar.Alvarez_error.sum() / (sahar.shape[0]-5))**0.5

## Fit equation via least squares

In [ ]:
def chave2005(X, a, b, c, d):
    dap, den = X
    out = a + b * np.log(dap) + c * np.log(dap)**2 + d * np.log(dap)**3 + np.log(den)
    return out

def chave2014(X, a, b, c, d, e):
    E, den, dap = X
    out = a + b * E + c * np.log(den) + d * np.log(dap) + e * np.log(dap)**2
    return out

def chave2014_dh(X, a, b):
    dap, hei, den = X
    out = np.log(a) + b * (np.log(den) + np.log(hei) + 2 * np.log(dap))
    return out

In [ ]:
curve_fit(chave2005, (har.DBH_cm, har.Gravity), np.log(har.AGB_kg))

In [ ]:
har['ChaveI_new'] = np.exp(chave2005((har.DBH_cm, har.Gravity), -3.350, 3.688, -0.296, 0.026))
har['ChaveI_new_error'] = (np.log(har.ChaveI_new) - np.log(har.AGB_kg))**2
cf = har.ChaveI_new_error.sum() / (har.shape[0]-4)
print "SEE:", cf ** 0.5
print "CF:", np.exp(cf / 2)
print "log CF:", cf / 2

In [ ]:
curve_fit(chave2014, (har.E, har.Gravity, har.DBH_cm), np.log(har.AGB_kg))

In [ ]:
har['ChaveII_new'] = np.exp(chave2014((har.E, har.Gravity, har.DBH_cm), -2.109, -0.896,  0.923,  2.794, -0.046))
har['ChaveII_new_error'] = (np.log(har.ChaveII_new) - np.log(har.AGB_kg))**2
cf = har.ChaveII_new_error.sum() / (har.shape[0]-5)
print "SEE:", cf ** 0.5
print "CF:", np.exp(cf / 2)
print "log CF:", cf / 2

In [ ]:
curve_fit(chave2014_dh, (har.DBH_cm, har.Total_height_m, har.Gravity), np.log(har.AGB_kg))

In [ ]:
har['ChaveII_dh_new'] = np.exp(chave2014_dh((har.DBH_cm, har.Total_height_m, har.Gravity), 0.06311, 0.9759))
har['ChaveII_dh_new_error'] = (np.log(har.ChaveII_dh_new) - np.log(har.AGB_kg))**2
cf = har.ChaveII_dh_new_error.sum() / (har.shape[0]-2)
print "SEE:", cf ** 0.5
print "CF:", np.exp(cf / 2)
print "log CF:", cf / 2

## Maximum likelihood fit - Spicy

In [ ]:
from scipy.optimize import minimize

In [ ]:
def logLike(true, guess):
    n = len(true)
    error = true - guess
    sigma = np.std(error)
    f = -(n/2.0)*np.log(2*np.pi) - (n/2.0) * np.log(sigma**2) - \
            (1.0/(2*sigma**2) * np.dot(error.T,error))
    return f

def opt_chave2005(pars):
    y = pars[0] + pars[1] * np.log(har.DBH_cm) + pars[2] * np.log(har.DBH_cm)**2 + \
            pars[3] * np.log(har.DBH_cm)**3 + np.log(har.Gravity)
    log = logLike(np.log(har.AGB_kg), y)
    return -1 * log

def opt_chave2014(pars):
    y = pars[0] + pars[1] * har.E + pars[2] * np.log(har.Gravity) + pars[3] * np.log(har.DBH_cm) \
        + pars[4] * np.log(har.DBH_cm)**2 
    log = logLike(np.log(har.AGB_kg), y)
    return -1 * log

### Chave 2005

In [ ]:
#coeffs = np.array([-3.350, 3.688, -0.296, 0.026])
coeffs = np.array([-2.350, 2.688, -1.296, 0.026])
#opt_chave2005(coeffs)
res = minimize(opt_chave2005, coeffs, method='BFGS')

In [ ]:
res

### Chave 2014

In [ ]:
#coeffs = np.array([-2.109, -0.896,  0.923,  2.794, -0.046])
coeffs = np.array([-3.109, -1.896,  1.923,  1.794, 1.046])
res = minimize(opt_chave2014, coeffs, method='BFGS')

In [ ]:
res

## Maximum likelihood fit
### equation Chave I

In [ ]:
import pymc3

In [ ]:
mymodel = pymc3.Model()
with mymodel:
    #a + b * np.log(dap) + c * np.log(dap)**2 + d * np.log(dap)**3 + np.log(den)
    a = pymc3.Uniform('a')
    b = pymc3.Uniform('b')
    c = pymc3.Uniform('c')
    d = pymc3.Uniform('d')
    #sigma = pymc3.Normal('sigma', mu=0, sd=0.4)
    
    y_exp = a + b * np.log(har.DBH_cm) + c * np.log(har.DBH_cm)**2 + d * np.log(har.DBH_cm)**3 + \
        np.log(har.Gravity)
        
    Y_obs = pymc3.Normal('Y_obs', mu=y_exp, sd=0.4, observed=np.log(har.AGB_kg))
    
    trace = pymc3.sample(5000, njobs=2)

In [ ]:
pymc3.summary(trace)

In [ ]:
map_estimate = pymc3.find_MAP(model=mymodel)

In [ ]:
map_estimate
# 0.1606, 1, 0.262888, 0 